In [ ]:
import pandas as pd
#from source import image_id_converter as img_idc
import matplotlib.pyplot as plt
from PIL import Image
#import numpy as np
#import pandas as pd
import pathlib as pl
from pathlib import Path
import os

In [ ]:
import ollama
#import json
#import re

In [ ]:
# Get path to current directory: 
project_path = Path.cwd()

# Define path to folder with images: 
image_dir = project_path/'visual_genome_proc_data'  # Replace with your directory containing images


### LLM for image analysis first try:

In [ ]:


# For batch processing your images
def analyze_image(image_path, prompt):
    response = ollama.chat(
        # Choose the LLM you want to use: 
        
        #model="minicpm-v",  # or "llava"
        #model="llama3.2-v",
        model="llama3.2-vision:90b",
        messages=[{
            'role': 'user', 
            'content': prompt,
            'images': [image_path]
        }]
    
    )
    return response['message']['content']

# Example prompt: 
prompt = "List the 5 most prominent objects in this image and categorize the image type (photography, map, painting, etc.)"



In [ ]:
# Get list of all files in directory
image_files = os.listdir(image_dir)
# Choose an image file: 
image_file_path = image_dir / image_files[3]
image_file_path

In [ ]:
# Show image: 
image = Image.open(image_file_path)
plt.imshow(image)

In [ ]:
# Ask the LLM about the image: 
analyze_image(image_file_path, prompt)